In [ ]:
# Use 'shift+enter' to execute each cell

# This script checks that your git repository is up-to-date and 
# makes sure that your enviroment is setup correctly
%cd ~/AWAKE_ANALYSIS_TOOLS/utilities
!./git_check.sh
%run setup.py
%cd ~/AWAKE_ANALYSIS_TOOLS/scratch

In [ ]:
# If the output of the above cell says "Need to pull" *AND* you are sure you don't have
# any changes to your working directory (scratch excluded), execute the line below
!git pull

In [ ]:
# Set the path to the cut file
inputFile=os.environ['AAT']+'scratch/cutInput_example.txt'
print('Loaded Input File with following parameters:\n')
!cat {inputFile}

In [ ]:
# Parse the cut file and get list of files in specified time range
InputParsed=cp.inputParser(inputFile)
print(str(len(InputParsed.flist))+' events in this time range.')

In [ ]:
# Return a list of files satisfying the cuts
print('Applying Cuts . . .')
start = time.time()
((fb,null),fl) = InputParsed()
file_bool = np.array(fb)
file_list = np.array(fl)
use_files = file_list[file_bool]
end = time.time()
print(str(len(use_files))+' events passed cuts.')
print('Time Elapsed = '+str(end-start))

In [ ]:
# Now create the ntuple
start = time.time()
cn.createNtuples(use_files,InputParsed)
end = time.time()
print('Time Elapsed = '+str(end-start))

In [ ]:
# Load the ntuple and get streak data
from returnData import returnData

f = h5py.File('example_ntup.h5')
date_set = 'file_info/FileTime'
strk_img_set = 'AwakeEventData/XMPP-STREAK/StreakImage/streakImageData'
strk_wid_set = 'AwakeEventData/XMPP-STREAK/StreakImage/streakImageWidth'
strk_ax_set = 'AwakeEventData/XMPP-STREAK/StreakImage/streakImageTimeValues'
bl_temp_set = 'AwakeEventData/awake.XAWAVS_BEAML_TT_AVG/PosSt/value'
df_temp_set = 'AwakeEventData/awake.XAWAVS_DF_TT_AVG/PosSt/value'
uf_temp_set = 'AwakeEventData/awake.XAWAVS_UF_TT_AVG/PosSt/value'
vx_temp_set = 'AwakeEventData/awake.XAWAVS_VX_TT_AVG/PosSt/value'

date_data = returnData(f,date_set)/1e9
img_data = returnData(f,strk_img_set)
img_width = returnData(f,strk_wid_set)
time_data = returnData(f,strk_ax_set)
bl_temp = returnData(f,bl_temp_set)
df_temp = returnData(f,df_temp_set)
uf_temp = returnData(f,uf_temp_set)
vx_temp = returnData(f,vx_temp_set)

# Correct the time axis
time_axis = time_data[:,1]
time_axis[0]=2*time_axis[1]-time_axis[2]
x_axis = np.linspace(-2,2,img_width[0])

# Set ROI and image analysis parameters
roi = {}
roi['x_min'] = -1.5
roi['x_max'] = 1.5
roi['y_min'] = 0
roi['y_max'] = 200
do = {}
do['streak'] = 2.5
cam = 'XMPP_STREAK'

# Check to make sure parameters are ok
img = img_data[:,:,1]
my_dict = analyze_frame(img,x_axis,time_axis,roi,[],do)
(n_x,n_t) = np.shape(my_dict['img'])
nfft = len(my_dict['streak_data']['fftb'])

print('Data Loaded')

In [ ]:
# Plot results in a loop
%matplotlib notebook
nEvts = len(img_width)
im_array = np.zeros((n_x,n_t,nEvts))
fft_array = np.zeros((nfft,nEvts))
avg_array = np.zeros((nfft,nEvts))
for i in range(nEvts):
    img = img_data[:,:,i]
    my_dict = analyze_frame(img,x_axis,time_axis,roi,[],do)
    if i==0:
        this_fft=my_dict['streak_data']['fftb']
        fft_avg=this_fft
    else:
        this_fft=my_dict['streak_data']['fftb']
        fft_avg+=this_fft
        fft_avg*=i/(i+1)
    if any(np.isnan(this_fft)):
        print(i)
    im_array[:,:,i] = my_dict['img']
    fft_array[:,i] = my_dict['streak_data']['fftb']
    avg_array[:,i] = fft_avg
    fig = view_streak(my_dict,cam,[-0.5, 0.5],[400, 900],fft_avg)
    fig.canvas.draw()
    time.sleep(0.02)
    if i < (nEvts-1):
        plt.clf()